In [1]:
%load_ext autoreload
%load_ext autotime

time: 414 µs (started: 2023-03-02 13:14:36 -08:00)


In [2]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.47 s (started: 2023-03-02 13:14:36 -08:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
from dask.distributed import wait

time: 1.07 s (started: 2023-03-02 13:14:37 -08:00)


In [4]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

# log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
# log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
#         force_local=True,
        cluster_settings=dict(
#             cores=8,
            dashboard_port=3616,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 587 ms (started: 2023-03-02 13:14:38 -08:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

['http://192.168.65.58:3616/status']

time: 3.98 ms (started: 2023-03-02 13:14:39 -08:00)


In [6]:
client = recorder_analyzer.cluster_manager.clients[0]

time: 742 µs (started: 2023-03-02 13:14:39 -08:00)


In [7]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

ValueError: Metadata inference failed in `set_delta`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KilledWorker("('aggregate-agg-29fbbe983763dcdd35d2e20741aeae79', 0)", <WorkerState 'tcp://192.168.64.23:43931', name: cluster-0-1-19, status: closed, memory: 0, processing: 1>)

Traceback:
---------
  File "/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/dask/dataframe/utils.py", line 177, in raise_on_meta_error
    yield
  File "/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/dask/dataframe/core.py", line 6001, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "../wisio/_recorder/analysis.py", line 234, in set_delta
    df[csp_col] 

time: 40min 25s (started: 2023-03-02 13:14:39 -08:00)


In [ ]:
views, max_io_time = res

In [ ]:
views[('trange',)]['expanded_view'].head()

In [ ]:
views[('trange',)]['grouped_view'].head()

In [ ]:
%autoreload 2
from wisio._recorder.analysis import compute_unique_filenames, compute_unique_processes

unique_filenames = compute_unique_filenames(log_dir=log_dir, unique_file_ids=[])
unique_processes = compute_unique_processes(log_dir=log_dir, unique_proc_ids=[])


In [ ]:
from wisio._recorder.bottlenecks import RecorderBottleneckGenerator

bottleneck_generator = RecorderBottleneckGenerator(
    log_dir=log_dir,
    views=views,
    view_types=['trange', 'file_id', 'proc_id'],
    unique_filenames=unique_filenames,
    unique_processes=unique_processes
)

bottlenecks = bottleneck_generator.generate_bottlenecks(max_io_time=max_io_time)

len(bottlenecks)


In [9]:
import itertools as it
from pathlib import PurePath


def filename(file_id):
    path = unique_filenames[file_id]['filename']
    pure_path = PurePath(path)
    # print(pure_path.parent.name)
    if pure_path.parent.name:
        return f"../{pure_path.name}"
    return pure_path.name


def io_cat_fmt(io_cat_min, io_cat_max):
    return ''.join([['-', 'r', 'w', 'm'][io_cat] for io_cat in set([io_cat_min, io_cat_max])]).rjust(3, '-')


def procname(proc_id):
    app = unique_processes[proc_id]['app']
    hostname = unique_processes[proc_id]['hostname']
    rank = str(unique_processes[proc_id]['rank'])
    return '-'.join(filter(None, [app, hostname, rank]))


def acc_pat_fmt(acc_pat_min, acc_pat_max):
    return 'seq.' if all(x == 0 for x in [acc_pat_min, acc_pat_max]) else 'rand.'


bokeh_df = last_llc_ddf.merge(last_llc_ddf_grp, on=('trange',), how='left', suffixes=(None, '_y'))
bokeh_df = bokeh_df.sort_values(by=[('duration_y', 'gsum'), ('duration', 'sum')], ascending=[False, False])
# bokeh_df[('filename',)] = bokeh_df[("filename",)].apply(filename)
bokeh_df['trange_fmt'] = bokeh_df['trange'].apply(lambda x: f"{str(x-1).rjust(5, '0')}-{str(x).rjust(5, '0')}")
bokeh_df['filename'] = bokeh_df['file_id', ''].apply(filename)
bokeh_df['procname'] = bokeh_df['proc_id', ''].apply(procname)
bokeh_df['io_cat_fmt'] = bokeh_df.apply(lambda x: io_cat_fmt(x[('io_cat', 'min')], x[('io_cat', 'max')]), axis=1)
bokeh_df['acc_pat_fmt'] = bokeh_df.apply(lambda x: acc_pat_fmt(x[('acc_pat', 'min')], x[('acc_pat', 'max')]), axis=1)
bokeh_df


trange              file_id              proc_id acc_pat       bw       \
                                                        max min  per  sum   
1     111  8153702741511540096   109922880417957963       0   0  0.0  0.0   
4     111    40765165787810657  6669293494323906635       0   0  0.0  0.0   
0     112  8682059482099919786   109922820288415819       0   0  0.0  0.0   
2     124 -1401937498136866036   109922137388615755       0   0  0.0  0.0   
3      85 -2370583934253578835  6669294061259589707       0   0  0.0  0.0   
5     102  8987176022018307730  3612046755538929739       0   0  0.0  0.0   
6     939  6142509188972423790  6669294061259589707       0   0  0.0  0.0   
7     109  4312208068175405484   988845911289366603       0   0  0.0  0.0   
8     118  6522048368339424215  6001848356685482059       0   0  0.0  0.0   
9     104 -6111213961795633975  7496344667563231307       0   0  0.0  0.0   
10    962  6142509188972423790  6669293494323906635       0   0  0.0  0.0   
11    119 -1637176655482389788  6691485070651295819       0   0  0.0  0.0   

              duration            ... size_y          xfer_y                   \
     sum_fmt       csp     delta  ...    sum sum_fmt max_fmt mean_fmt min_fmt   
1   0.00GB/s  0.002600  0.001289  ...      0  0.00GB    <4KB     <4KB    <4KB   
4   0.00GB/s  0.006429  0.001271  ...      0  0.00GB    <4KB     <4KB    <4KB   
0   0.00GB/s  0.001311  0.001311  ...      0  0.00GB    <4KB     <4KB    <4KB   
2   0.00GB/s  0.003884  0.001284  ...      0  0.00GB    <4KB     <4KB    <4KB   
3   0.00GB/s  0.005158  0.001274  ...      0  0.00GB    <4KB     <4KB    <4KB   
5   0.00GB/s  0.007653  0.001224  ...      0  0.00GB    <4KB     <4KB    <4KB   
6   0.00GB/s  0.008868  0.001214  ...      0  0.00GB    <4KB     <4KB    <4KB   
7   0.00GB/s  0.010080  0.001212  ...      0  0.00GB    <4KB     <4KB    <4KB   
8   0.00GB/s  0.011234  0.001153  ...      0  0.00GB    <4KB     <4KB    <4KB   
9   0.00GB/s  0.012383  0.001150  ...      0  0.00GB    <4KB     <4KB    <4KB   
10  0.00GB/s  0.013524  0.001141  ...      0  0.00GB    <4KB     <4KB    <4KB   
11  0.00GB/s  0.014622  0.001098  ...      0  0.00GB    <4KB     <4KB    <4KB   

     trange_fmt          filename                 procname io_cat_fmt  \
                                                                        
1   00110-00111   ../chr8.HG00278  individuals-lassen338-0        --m   
4   00110-00111   ../chr9.HG00278  individuals-lassen334-0        --m   
0   00111-00112   ../chr8.HG00278  individuals-lassen338-0        --m   
2   00123-00124   ../chr8.HG00278  individuals-lassen338-0        --m   
3   00084-00085   ../chr9.HG00278  individuals-lassen334-0        --m   
5   00101-00102   ../chr8.HG00278  individuals-lassen339-0        --m   
6   00938-00939                    individuals-lassen334-0        --m   
7   00108-00109   ../chr8.HG00278  individuals-lassen340-0        --m   
8   00117-00118  ../chr10.HG00278  individuals-lassen329-0        --m   
9   00103-00104   ../chr9.HG00278  individuals-lassen333-0        --m   
10  00961-00962                    individuals-lassen334-0        --m   
11  00118-00119  ../chr10.HG00278  individuals-lassen325-0        --m   

   acc_pat_fmt  
                
1         seq.  
4         seq.  
0         seq.  
2         seq.  
3         seq.  
5         seq.  
6         seq.  
7         seq.  
8         seq.  
9         seq.  
10        seq.  
11        seq.  

[12 rows x 63 columns]

time: 153 ms (started: 2023-01-12 23:29:38 -08:00)


In [10]:
from bokeh.io import show, save, output_file
from bokeh.models import (ColumnDataSource, DataCube, GroupingInfo, NumberFormatter,
                          StringFormatter, SumAggregator, TableColumn, CustomJS)

# output_file(filename='test.html')

source = ColumnDataSource(bokeh_df.reset_index())

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

formatter = StringFormatter(font_style='bold')
duration_formatter = NumberFormatter(format='0.000%')

columns = [
    TableColumn(field='procname_', title='Time Range > File ID > Proc ID', width=80, formatter=formatter),
    TableColumn(field='duration_per', title='Duration %', width=40,  formatter=duration_formatter),
    TableColumn(field='io_cat_fmt_', title='I/O Type', width=20),
    TableColumn(field='acc_pat_fmt_', title='Acc. Pat.', width=20),
    TableColumn(field='size_sum_fmt', title='I/O Size', width=30),
    TableColumn(field='bw_sum_fmt', title='BW', width=30),
    TableColumn(field='xfer_min_fmt', title='Min. Xfer', width=30),
    TableColumn(field='xfer_max_fmt', title='Max. Xfer', width=30),
]

grouping = [
    GroupingInfo(getter='trange_fmt_', aggregators=[SumAggregator(field_='duration_per')]),
    GroupingInfo(getter='filename_', aggregators=[SumAggregator(field_='duration_per')]),
]

cube = DataCube(source=source, columns=columns, grouping=grouping, target=target, sizing_mode='stretch_width')

show(cube)


time: 127 ms (started: 2023-01-12 23:29:39 -08:00)


In [11]:
trange_agg_cut_piv = hlm_dfs['trange'].reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,read,metadata
trange,,
1,188.209763,0.000000
2,362.724030,0.000000
3,432.858215,0.000000
4,447.976074,0.000000
5,474.298248,0.000000
...,...,...
981,0.000000,218.287857
982,0.000000,220.791077
983,0.000000,244.485245


time: 47.2 ms (started: 2023-01-12 23:29:40 -08:00)


In [15]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

stat_plot = figure(width=800, height=200, tools="xpan,xbox_zoom,xbox_select,reset", active_drag="xbox_select")
stat_plot.vbar_stack(impacts, width=0.7, x="trange", source=source, color=['red', 'blue'], legend_label=impacts)

stat_plot.y_range.start = 0
stat_plot.xgrid.grid_line_color = None
stat_plot.legend.location = "top_right"
stat_plot.legend.click_policy="mute"

show(stat_plot)

time: 221 ms (started: 2023-01-13 05:52:16 -08:00)
